In [1]:
import xarray as xr
from pyproj import Proj
from dask.diagnostics import ProgressBar
import numpy as np
import matplotlib.pyplot as plt
import xesmf as xe
import rioxarray

In [ ]:
version = '0.2'

In [ ]:
def open_elevation_data():
    data = rioxarray.open_rasterio('/data/AIDL-UPC/wc2.1_30s_elev.tif')
    data = data.rename({'x': 'lon','y': 'lat'})
    data = data.assign_coords(lat=-data.lat)
    return data

# compute mean and std of the ERA5 data

In [ ]:
vars_name  = ['pr', 'rh', 'tas', 'tasmax', 'tasmin', 'sfcWind'] # 'sp',, 'uas', 'vas'
path = '/home/vestella/data/NAS-data/terolink/archive/reanalysis/unversioned/era5/'

In [ ]:
lat_max = 720
for ivar in range(len(vars_name)):
    data_era5 = xr.open_mfdataset(path + vars_name[ivar]+'_day/'+vars_name[ivar] + '_*', engine = 'zarr' )
    data_era5 = data_era5.sel(time = slice(*['2000', '2018']))
    if ivar ==0:
        mean = np.empty((len(vars_name), lat_max, data_era5.lon.shape[0]), dtype = np.float32)
        std = np.empty((len(vars_name), lat_max, data_era5.lon.shape[0]), dtype = np.float32)
        mmean = np.empty((len(vars_name)), dtype = np.float64)
        sstd = np.empty((len(vars_name)), dtype = np.float64)
    if lat_max!= data_era5.lat.shape[0]:
        data_era5 = data_era5.isel(lat = np.arange(720))
    print(data_era5.lat.values[0], data_era5.lat.values[-1])
    mean_single = data_era5[vars_name[ivar]].mean(dim = 'time').values
    std_single = data_era5[vars_name[ivar]].std(dim = 'time').values
    mean_all = data_era5[vars_name[ivar]].mean()
    std_all = data_era5[vars_name[ivar]].std()
    
    mean[ivar, :,:] = mean_single
    std[ivar, :,:] = std_single
    mmean[ivar] = mean_all
    sstd[ivar] = std_all
    
da = xr.Dataset(
data_vars=dict(
    mean=([ "variable", "lat", "lon" ], mean),
    std=([ "variable", "lat", "lon" ], std),
    mean_all = ([ "variable"], mmean),
    std_all=([ "variable" ], sstd),
            ),
coords=dict(
    lon=(["lon"], data_era5.lon.values),
    lat=(["lat"], data_era5.lat.values),
    variable=vars_name,
),
attrs=dict(
    description="std and mean of the period 2000-2018 for each of the variables",
),
)

da.to_netcdf('/data/AIDL-UPC/std_mean_variables_ERA5_v'+version+'.nc')
print('/data/AIDL-UPC/std_mean_variables_ERA5_v'+version+'.nc')

# compute mean and std of the elevation data

In [ ]:
data = open_elevation_data()

In [ ]:
land_ocean = xr.where(data==data._FillValue,0,1)
elevation = xr.where(data==data._FillValue,0,data)
elevation_norm = (elevation-elevation.mean())/elevation.std()

In [ ]:
land_ocean.to_netcdf('/data/AIDL-UPC/land_ocean_mask_v'+version+'.nc')
print('/data/AIDL-UPC/land_ocean_mask_v'+version+'.nc')

elevation_norm.to_netcdf('/data/AIDL-UPC/elevation_norm_v'+version+'.nc')
print('/data/AIDL-UPC/elevation_norm_v'+version+'.nc')

elevation.to_netcdf('/data/AIDL-UPC/elevation_v'+version+'.nc')
print('/data/AIDL-UPC/elevation_v'+version+'.nc')